In [1]:
import pandas as pd


In [7]:
df = pd.read_csv('data/results.csv',sep='\t')

In [9]:
df=df.set_index(['Algo','Dataset Type','Group']).dropna()

In [10]:
df.columns = df.columns.str.replace(' ', '')

In [11]:
group_counts=df[['Count']].drop(level ='Group',index='All')

In [12]:
m_f_results_tmp=df[['recall_avg','user_success_avg','map_at_2','map_at_5','map_at_10','map_at_20']]

In [13]:
weighted_average = group_counts.stack().droplevel(3).reindex(m_f_results_tmp.stack().sort_index().index,method='ffill')

In [ ]:
m_f_results_tmp

In [15]:
m_f_results_tmp=m_f_results_tmp.drop(level ='Group',index='All')

In [16]:
weighted_average=group_counts.stack().droplevel([0,3]).drop_duplicates().reindex(m_f_results_tmp.stack().index,method='ffill').mul(m_f_results_tmp.stack())

In [19]:
total_sum= weighted_average.unstack(2).transpose().sum()

In [21]:
# Lastly, normalizing the summed values by the total_sum! (we fill with 0 where the normalizing factor was 0)
compounding_factors = weighted_average.unstack(2).div(total_sum,axis='index').fillna(0)

In [ ]:
compounding_factors

In [23]:
compounding_factors_tmp=compounding_factors.stack().swaplevel(2,3).sort_index()

In [ ]:
compounding_factors_tmp

In [25]:
groups_percentages=group_counts.stack().unstack(2).div(group_counts.stack().unstack(2).sum(axis='columns'),axis='index').stack().droplevel(2).sort_index()

In [ ]:
groups_percentages

In [27]:
groups_percentages_tmp=groups_percentages.reindex_like(compounding_factors_tmp,method='ffill')

In [ ]:
# Showing m and f in the axis
# RUN ONLY ONCE!
groups_percentages_tmp = groups_percentages_tmp.unstack(2).transpose()
compounding_factors_tmp = compounding_factors_tmp.unstack(2).transpose()
compounding_factors_tmp

In [29]:
import numpy as np

In [30]:
def kl_event_diff(p, q):
    return - (p.loc['M'] * np.log2(p.loc['M']/q.loc['M'])) + (p.loc['F'] * np.log2(p.loc['F']/q.loc['F']))

In [31]:
# Combining the two
kl_diffs = groups_percentages_tmp.combine(compounding_factors_tmp,overwrite=False,func=kl_event_diff)


In [32]:
kl_diffs= kl_diffs.drop_duplicates().stack([0,1,2]).droplevel(0).unstack(2)

In [250]:
!pip install xlwt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 701.6 kB/s eta 0:00:00a 0:00:01


In [33]:
kl_diffs.to_excel('Comft.xls')

/tmp/ipykernel_2835154/2297611472.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  kl_diffs.to_excel('Comft.xls')


In [ ]:
males_compounding_factors = compounding_factors['M'].unstack().sort_index()
females_compounding_factors = compounding_factors['F'].unstack().sort_index()
males_compounding_factors

In [ ]:
females_compounding_factors

Difference

In [ ]:
m_f_results_tmp.stack().unstack(2)['M'].unstack()

In [37]:
m_f_results_tmp
males_means = m_f_results_tmp.stack().unstack(2)['M'].unstack()
females_means = m_f_results_tmp.stack().unstack(2)['F'].unstack()
diff_means = (males_means - females_means)

In [38]:
diff_means.to_excel('Recgap.xls')

/tmp/ipykernel_2835154/140047849.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  diff_means.to_excel('Recgap.xls')
